# Outline
Text goes here

In [100]:
# Imports
import pandas as pd
from citipy import citipy
import random
import requests

# Get API key from api_keys.py
import api_keys
api_ow = api_keys.api_ow

In [101]:
# Sample API call for OpenWeather geolocation:
# http://api.openweathermap.org/geo/1.0/direct?q=CITYNAME,STATECODE,COUNTRYCODE&limit=LIMIT&appid=APIKEY

# Sample API call for OpenWeather weather data:
# units = "imperal"
#api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + LATITUDE + "&lon=" + LONGITUDE + "&appid=" + api_ow + "&units=" + units

In [107]:
# Functions

# Generates a random number between -180 and 180 degrees (0 is Greenwich, England).
def genLatitude():
    randomLat = random.uniform(-180, 180)
    return round(randomLat, 5)

# Generates a random number between -90 and 90 degrees (0 is equator)
def genLongitude():
    randomLong = random.uniform(-90, 90)
    return round(randomLong, 5)

# Send an OpenWeather API request with a city name, a country code, and a limit of the number of responses. In most cases the limit should be 1.
# Return the details of the city in a dataframe
def getCityCoordinates(city, country, limit):
    api_string = "http://api.openweathermap.org/geo/1.0/direct?q=" + city + "," + country + "&limit=" + str(limit) + "&appid=" + api_ow + "&lang=en"
    result = requests.get(api_string)
    #print("Sending API request: " + api_string)
    #print("Got API response: " + result.content.decode("utf-8"))
    df_result = pd.read_json(result.content.decode("utf-8"))
    result_columns = ["name", "lat", "lon", "country", "state"]
    df_result = df_result.loc[:, result_columns]
    return df_result

# Send an OpenWeather API request with a set of latitude and longitude coordinates and the type of units (valid values are "standard", "metric", "imperial").
# Response will be in JSON. Extract temperature, humidity, cloudiness, wind speed.
# Return a single-row dataframe.
def getWeatherData(lat, long, units):
    api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + lat + "&lon=" + long + "&appid=" + api_ow + "&units=" + units
    result = requests.get(api_string)
    #result_json = pd.read_json(result)
    #print(result_json)
    

In [103]:
# Generate 500 random sets of long/lat coordinates.
# Get the nearest city to each coordinates. If it's not in the list, add its weather data.
cities_500 = []
count = 1
while count <= 500:
    lat = genLatitude()
    long = genLongitude()
    nearest = citipy.nearest_city(lat, long)
    city = [nearest.city_name, nearest.country_code]

    # Only add the city if it's unique
    if city not in cities_500:
        cities_500.append(city)
        
        count += 1

In [104]:
# Dataframe to hold city data.
df_cities = pd.DataFrame(columns=["City Name", "Latitude", "Longitude", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind speed (MPH)"])
df_cities

,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind speed (MPH)


In [110]:
df_test = getCityCoordinates("iqaluit", "ca", 1)

C:\Users\jackl\AppData\Local\Temp\ipykernel_23600\2721219283.py:20: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_result = pd.read_json(result.content.decode("utf-8"))


In [111]:
df_test

,name,lat,lon,country,state
0,Iqaluit,63.74944,-68.521857,CA,Nunavut
